In [1]:
import numpy as np
from medmnist import PneumoniaMNIST
import torch
import random
import copy

from skimage.io import imshow
import neptune
run = neptune.init_run(project='momkeybomkey/Federated',
                       api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjNDdlN2ZhNy00ZmJmLTQ4YjMtYTk0YS1lNmViZmZjZWRhNzUifQ=='
                       )

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/momkeybomkey/Federated/e/FED-20


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
def _collate_fn(data):
        xs = []

        for x, _ in data:
            x = np.array(x).astype(np.float32) / 255.
            xs.append([x])

        return np.array(xs)

def shuffle_iterator(iterator):
    # iterator should have limited size
    index = list(iterator)
    total_size = len(index)
    i = 0
    random.shuffle(index)
    result = []

    while len(result) < total_size:
        result.append(index[i])
        i += 1
        
        if i >= total_size:
            i = 0
            random.shuffle(index)

    return result

def get_loader(dataset, random):
    total_size = len(dataset)
    print('Size', total_size)
    if random:
        index_generator = shuffle_iterator(range(total_size))
    else:
        index_generator = list(range(total_size))

    while True:
        data = []

        for _ in range(len(index_generator)):
            idx = index_generator.pop()
            data.append(dataset[idx])

        return _collate_fn(data)

In [4]:
def generate_noise(data, noise_factor = 0.2):
    data += noise_factor * np.random.normal(size=data.shape)
    data = np.clip(data, 0., 1.)
    return data

In [5]:
def even_data(split):
    min_size = min([len(data) for data in split])
    return [data[:min_size] for data in split]

In [6]:
def split_data(data, n, noise_factor = 0.1):
    m = len(data)
    data_copy = copy.deepcopy(data)
    clean_split = []
    
    for i in range(n):
        clean_split.append(data_copy[m * i // n: m * (i + 1) // n])

    clean_split = even_data(clean_split)
    noisy_split = copy.deepcopy(clean_split)
    noisy_split = [generate_noise(x, noise_factor) for x in noisy_split]

    clean_split = np.clip(np.array(clean_split), 0., 1.)
    noisy_split = np.clip(np.array(noisy_split), 0., 1.)
    
    clean_split = torch.tensor(clean_split).float()
    noisy_split = torch.tensor(noisy_split).float()

    clean_split = clean_split.to(device)
    noisy_split = noisy_split.to(device)

    return clean_split, noisy_split

In [ ]:
train = PneumoniaMNIST(split="train", download=True, size=128)
test = PneumoniaMNIST(split="test", download=True, size=128)
val = PneumoniaMNIST(split="val", download=True, size=128)

train_loader = get_loader(train, random=True)
test_loader = get_loader(test, random=True)

val_loader = get_loader(val, random=False)

val_clean, val_noisy = split_data(val_loader, 1)

Using downloaded and verified file: C:\Users\malth\.medmnist\pneumoniamnist_128.npz
Using downloaded and verified file: C:\Users\malth\.medmnist\pneumoniamnist_128.npz
Using downloaded and verified file: C:\Users\malth\.medmnist\pneumoniamnist_128.npz
Size 4708
Size 624
Size 524


In [8]:
val_clean.size()

torch.Size([1, 524, 1, 128, 128])

In [9]:
loss_fn = torch.nn.MSELoss()

In [10]:
def PSNR(y_true, y_pred):
    return 20 * torch.log10(torch.max(y_true) / loss_fn(y_true, y_pred))

In [21]:
def neptune_log(epoch, model, loss, psnr):
    print("Evaluation")
    run["evaluation/mse"].append(loss)
    run["evaluation/psnr"].append(psnr) 
    run[f"images/reconstructed_{epoch + 1}"].upload(neptune.types.File.as_image(model(val_noisy[0][0])[0].cpu().detach().numpy()))
    print("")

In [22]:
def neptune_val_images(model):
    print("Final evaluation")
    
    for i in range(1, 6):
        run[f"validation/original_{i}"].upload(neptune.types.File.as_image(val_clean[0][i][0].cpu().detach().numpy()))
        run[f"validation/noisy_{i}"].upload(neptune.types.File.as_image(val_noisy[0][i][0].cpu().detach().numpy()))
        run[f"validation/reconstructed_{i}"].upload(neptune.types.File.as_image(model(val_noisy[0][i])[0].cpu().detach().numpy()))

In [13]:
# Limited data
# train_load = split_data(train_load, 5)
# test_load = split_data(test_load, 5)

# train_load = train_load[0][0]
# test_load = test_load[0][0]

In [23]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, 3, stride=2, padding=1),
            torch.nn.Flatten(0, -1),
            torch.nn.Linear(64 * 8 * 8, 128)
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(128, 64 * 8 * 8),
            torch.nn.Unflatten(0, (64, 8, 8)),
            torch.nn.ConvTranspose2d(64, 64, 3, stride=2, padding=1, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [30]:
def run_model(n, epochs):
    # Make overall model
    autoencoder = AE()
    autoencoder.to(device)

    try:
        autoencoder.load_state_dict(torch.load("./Models/model_test.pth"))
    except:
        pass

    # neptune_log(-1, autoencoder)
    run["images/original"].upload(neptune.types.File.as_image(val_clean[0][0][0].cpu().numpy()))
    run["images/noisy"].upload(neptune.types.File.as_image(val_noisy[0][0][0].cpu().numpy()))

    # Make n models
    models = [AE() for _ in range(n)]
    

    # Split data
    train, train_noisy = split_data(train_loader, n)
    test, test_noisy = split_data(test_loader, n)

    # Get central weights
    primary_weights = autoencoder.state_dict()
    for model in models:
        model.to(device)
        model.load_state_dict(primary_weights)
    
    # Train the networks
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}")

        for i, model in enumerate(models):
            print(f"Autoencoder {i + 1}")
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

            total_loss = 0
            total_psnr = 0

            for j in range(len(train[i])):
                optimizer.zero_grad()
                loss = loss_fn(model(train_noisy[i][j]), train[i][j])
                loss.backward()
                optimizer.step()
            
            for noisy, clean in zip(test_noisy[i], test[i]):
                loss = loss_fn(model(noisy), clean)
                total_loss += loss.item()
                total_psnr += PSNR(model(noisy), clean).item()

            total_loss /= len(test[i])
            total_psnr /= len(test[i])
            run[f"evaluation/autoencoder_{i + 1}/mse"].append(total_loss)
            run[f"evaluation/autoencoder_{i + 1}/psnr"].append(total_psnr)
            print(f"Loss {total_loss}")
            print(f"PSNR {total_psnr}")

        # Aggregate the models
        for key in primary_weights.keys():
            primary_weights[key] = sum([model.state_dict()[key] for model in models]) / n

        for model in models:
            model.load_state_dict(primary_weights)

        autoencoder.load_state_dict(primary_weights)

        print(f"Epoch {epoch + 1} completed")
        
        total_loss = 0
        total_psnr = 0

        for noisy, clean in zip(val_noisy[0], val_clean[0]):
            loss = loss_fn(autoencoder(noisy), clean)
            total_loss += loss.item()
            total_psnr += PSNR(autoencoder(noisy), clean).item()
        
        total_loss /= len(val_noisy[0])
        total_psnr /= len(val_noisy[0])

        print(f"Validation Loss {total_loss}")
        print(f"Validation PSNR {total_psnr}")
        print("")

        neptune_log(epoch, autoencoder, total_loss, total_psnr)

        torch.save(autoencoder.state_dict(), "./Models/model_test.pth")
    
    neptune_val_images(autoencoder)

    return autoencoder

In [31]:
autoencoder = run_model(5, 600)

C:\Users\malth\AppData\Local\Temp\ipykernel_3580\2248712902.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoencoder.load_state_dict(torch.load("./Models/model_test.p

KeyboardInterrupt: 

In [ ]:
# run.stop()